# SURFRAD and SOLRAD
The SURFRAD and SOLRAD are two monitoring networks in the United States operated by the National Oceanic and Atmospheric Administration (NOAA). 

In [4]:
def convert_latitude_longitude(l):
    """Function to convert from latitude/longitude string
    to float with sign convention of ISO 19115"""
    l = l.replace(' ', '')  # Remove all spaces
    cardinal_signs = {'N': 1, 'S': -1, 'E': 1, 'W': -1}
    for c in cardinal_signs.keys():
        if c in l:
            return cardinal_signs[c]*float(l.replace(c, '').replace('°', ''))

In [5]:
surfrad_url = 'https://www.esrl.noaa.gov/gmd/grad/surfrad/sitepage.html'
surfrad_stations = pd.read_html(surfrad_url, index_col=0)[0].iloc[:-1]
surfrad_stations[['Latitude', 'Longitude']] = surfrad_stations[['Latitude', 'Longitude']].applymap(convert_latitude_longitude)
surfrad_stations

,Name,Latitude,Longitude,Elevation,Time Zone,Installed
Code,,,,,,
BND*,"Bondville, Illinois",40.05192,-88.37309,230 m,6 hours from UTC,April 1994
TBL,"Table Mountain, Boulder, Colorado",40.12498,-105.23680,1689 m,7 hours from UTC,July 1995
DRA,"Desert Rock, Nevada",36.62373,-116.01947,1007 m,8 hours from UTC,March 1998
FPK,"Fort Peck, Montana",48.30783,-105.10170,634 m,7 hours from UTC,November 1994
GWN,"Goodwin Creek, Mississippi",34.25470,-89.87290,98 m,6 hours from UTC,December 1994
PSU,"Penn. State Univ., Pennsylvania",40.72012,-77.93085,376 m,5 hours from UTC,June 1998
SXF,"Sioux Falls, South Dakota",43.73403,-96.62328,473 m,6 hours from UTC,June 2003
SGP,"ARM Southern Great Plains Facility, Oklahoma",36.60406,-97.48525,314 m,6 hours from UTC,NaN


In [6]:
solrad_url = 'https://www.esrl.noaa.gov/gmd/grad/solrad/solradsites.html'
solrad_stations = pd.read_html(solrad_url, index_col=0)[0]
solrad_stations.loc['STE',['Name','Latitude','Longitude']] = ['Sterling, Virginia', '38.97203° N', '77.48690° W']
solrad_stations[['Latitude', 'Longitude']] = \
    solrad_stations[['Latitude', 'Longitude']].applymap(convert_latitude_longitude)
solrad_stations

,Name,Latitude,Longitude,Elevation
Code,,,,
ABQ,"Albuquerque, New Mexico",35.03796,-106.62211,1617 m
BIS,"Bismarck, North Dakota",46.77179,-100.75955,503 m
HNX,"Hanford, California",36.31357,-119.63164,73 m
MSN,"Madison, Wisconsin",43.07250,-89.41133,271 m
ORT,"Oak Ridge, Tennessee",35.96101,-84.28838,334 m
SLC,"Salt Lake City, Utah",40.77220,-111.95495,1288 m
SEA,"Seattle, Washington",47.68685,-122.25667,20 m
STE,"Sterling, Virginia",38.97203,-77.48690,85 m
TLH,"Tallahassee, Florida",30.39675,-84.32955,18 m


In [7]:
### Initialize Folium map
m = folium.Map(
    location=[40, -95],
    zoom_start=4,
    min_zoom=2,
    max_bounds=True,
    tiles='OpenStreetMap',
)

# Add stations to the map
for i, stations in enumerate([surfrad_stations, solrad_stations]):
    for index, row in stations.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{row['Name']} ({row.name})",
            icon=folium.Icon(color=['blue', 'red'][i])
            ).add_to(m)

# Add Category Legend
legend_html = """
<div style="position:fixed;
     bottom: 50px; 
     left: 50px; 
     width: 120px; 
     height: 105px; 
     border:2px solid grey; 
     z-index: 9999;
     font-size:14px;">
     &nbsp;<b>Station network:</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:blue"></i>&nbsp;SURFRAD<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp;SOLRAD<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp;SRML<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp;NREL<br>
</div>"""

m.get_root().html.add_child(folium.Element(legend_html))  # Add Legend

m.add_child(folium.LatLngPopup())  # Show latitude,longitude when clicking
m  # Show map